# P-multigrid Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

In [ ]:
# setup
finep = 3
coarsep = 2
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

finebasis = TensorH1LagrangeBasis(finep, finep + 1, dimension)
coarsebasis = TensorH1LagrangeBasis(coarsep, finep + 1, dimension)
ctofbasis = TensorH1LagrangeBasis(coarsep, finep, dimension, lagrangequadrature=true)

function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end

# fine grid diffusion operator
fineinputs = [
    OperatorField(finebasis, [EvaluationMode.gradient]),
    OperatorField(finebasis, [EvaluationMode.quadratureweights]),
]
fineoutputs = [OperatorField(finebasis, [EvaluationMode.gradient])]
finediffusion = Operator(diffusionweakform, mesh, fineinputs, fineoutputs)

# coarse grid diffusion operator
coarseinputs = [
    OperatorField(coarsebasis, [EvaluationMode.gradient]),
    OperatorField(coarsebasis, [EvaluationMode.quadratureweights]),
]
coarseoutputs = [OperatorField(coarsebasis, [EvaluationMode.gradient])]
coarsediffusion = Operator(diffusionweakform, mesh, coarseinputs, coarseoutputs)

# Jacobi smoother
jacobi = Jacobi(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, jacobi, [ctofbasis])

In [ ]:
# compute high frequency operator symbols
numberruns = 250
θ_min = -π/2
θ_max = 3π/2
θ_minhigh = π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [0.838]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        if θ[1] > θ_minhigh
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = max(eigenvalues...)
        end
    end

    # plot
    plot(maxeigenvalues, title="P-Multigrid Symbols")
    ylims!(0.0, max(maxeigenvalues...))
# -- 2D --
elseif dimension == 2
    # setup
    ω = [1.0]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        if θ[1] > θ_minhigh || θ[2] > θ_minhigh
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = max(eigenvalues...)
        end
    end

    # plot
    heatmap(maxeigenvalues, title="P-Multigrid Symbols", transpose=true)
end

In [ ]:
# compute low frequency operator symbols
numberruns = 250
θ_min = -π/2
θ_max = π/2

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [0.838]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns)

    # compute
    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        A = computesymbols(multigrid, ω, v, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i] = max(eigenvalues...)
    end

    # plot
    plot(maxeigenvalues, title="P-Multigrid Symbols")
    ylims!(0.0, max(maxeigenvalues...))
# -- 2D --
elseif dimension == 2
    # setup
    ω = [1.0]
    v = [1, 1]
    maxeigenvalues = zeros(numberruns, numberruns)

    # compute
    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        A = computesymbols(multigrid, ω, v, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i, j] = max(eigenvalues...)
    end

    # plot
    heatmap(maxeigenvalues, title="P-Multigrid Symbols", transpose=true)
end